In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from pprint import pprint
from utils import *

tagger = MeCab.Tagger()

D:\anaconda3\envs\data_analysis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
input_data_path = '../data/input_data/'
output_data_path = '../data/output_data/'
outer_data_path = '../data/outer_data/'

In [4]:
with open(input_data_path+'train_aozora_data.json') as f:
    novels_json = json.load(f)

novel_id_l = []
author_l = []
for author_data in novels_json:
    for novel in author_data['novels']:
        novel_id_l.append(novel['content_id'])
        author_l.append(author_data['author_id'])

In [7]:
model_path = '../data/outer_data/chive-1.2-mc5.kv'
w2v_model = KeyedVectors.load(model_path)

In [22]:
with open(input_data_path+'aozora_contents.json', encoding='utf-8') as f:
    novel_text_dic = json.load(f)

train_l = []
i = 0
for novel_content_id, author_id in zip(novel_id_l, author_l):
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)
    train_l.append(author_id+text_vec)
    print('\r'+str(i), end='')
    i+=1

train_df = pd.DataFrame(train_l, columns=['author_id', [i for i in range(len(text_vec))]])

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [21]:
train_df.loc[1]

text_vec     [-172.39893, -224.3913, 796.50244, -173.6843, ...
author_id                                            auth_0000
Name: 1, dtype: object

In [13]:
with open(output_data_path+'train_df.pickle', 'wb') as f:
    pickle.dump(train_df, f)